##### Report Serializer

Simultaneously updates the two storage layers: the filesystem and the database. It takes all the info from a report (inputs, parameters, and computed results) and puts it in the folder, and populates the SQLite database with the report metadata.

It receives all the context needed to fully describe a report:

- db_path and reports_root: (filesystem and database locations respectively)
- report_id: unique report identifier
- subject_id, module_id, and marker_id: identify what was analyzed
- requested_at: timestamp of request
- timeframe, zone_boundaries, and fitting: input parameters sent with original API request
- trajectory_result: full output dict returned by compute_trajectory()

Step 1: Writing the full report to the filesystem:

A subject-specific subdirectory is constructed under reports_root if it does not already exist. A report_payload dict is then assembled into one self-contained document and gets written to disk. The file is named after the report_id so that each report has a unique file. (indent=2 makes it human-readable).

Step 2: Insert metadata row into database

Only the metadata (not full result) goes into the database. "?" are placeholders preventing SQL injection. The "with get_connection(...) as conn" syntax uses the SQLite connection as context manager, automatically commiting the transaction on exit. 

In [ ]:
import json
import os

from core.state_store.database import get_connection

# all context needed to describe a report
def save_timegraph_report(
    db_path: str,
    reports_root: str,
    report_id: str,
    subject_id: str,
    module_id: str,
    marker_id: str,
    requested_at: str,
    timeframe: dict,
    zone_boundaries: dict,
    fitting: dict,
    trajectory_result: dict,
) -> None:
    # 1. Write the full report JSON to the filesystem
    report_dir = os.path.join(reports_root, subject_id)
    os.makedirs(report_dir, exist_ok=True)

    report_payload = {
        "report_id":       report_id,
        "subject_id":      subject_id,
        "module_id":       module_id,
        "marker_id":       marker_id,
        "requested_at":    requested_at,
        "timeframe":       timeframe,
        "zone_boundaries": zone_boundaries,
        "fitting":         fitting,
        "result":          trajectory_result,
    }

    report_path = os.path.join(report_dir, f"{report_id}.json")
    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(report_payload, f, indent=2)

    # 2. Insert metadata row into timegraph_reports
    with get_connection(db_path) as conn:
        conn.execute(
            """
            INSERT INTO timegraph_reports (
                report_id,
                subject_id,
                marker_id,
                module_id,
                requested_at,
                timeframe_from,
                timeframe_to,
                polynomial_degree,
                healthy_min,
                healthy_max,
                vulnerability_margin
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                report_id,
                subject_id,
                marker_id,
                module_id,
                requested_at,
                timeframe["from"],
                timeframe["to"],
                fitting["polynomial_degree"],
                zone_boundaries["healthy_min"],
                zone_boundaries["healthy_max"],
                zone_boundaries["vulnerability_margin"],
            ),
        )


ModuleNotFoundError: No module named 'core'